In [1]:
import hoomd
import numpy as np
# import ex_render

from monk import mimse, pair, prep

In [2]:
seed = 1000
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu, seed=seed)
N = 256
rng = prep.init_rng(seed + 1)
L = prep.len_from_phi(N, 1.2, dim=2)
snap = prep.approx_euclidean_snapshot(N, L, rng, dim=2, ratios=[60,40], diams=[1.0, 0.88])

sim.create_state_from_snapshot(snap)
# sim.create_state_from_gsd()

tree = hoomd.md.nlist.Tree(0.2)

integrator = hoomd.md.Integrator(dt=0.0025)
lang = hoomd.md.methods.Langevin(hoomd.filter.All(), 1.5)
lj = pair.KA_LJ(tree)
integrator.forces = [lj]
integrator.methods = [lang]
sim.operations.integrator = integrator

sim.run(0)
sim.state.thermalize_particle_momenta(hoomd.filter.All(), 1.5)

sim.run(1000)

sim.operations.integrator = None

integrator.forces.pop()
del integrator, lang

In [ ]:
hoomd.write.GSD.write(sim.state, "test.gsd")

In [3]:
fire = hoomd.md.minimize.FIRE(0.0025,
                              force_tol=1e-2,
                              angmom_tol=1e-2,
                              energy_tol=1e-7)

mimse_force = mimse.MIMSEForce(sim.state.box)
nve = hoomd.md.methods.NVE(hoomd.filter.All())
fire.forces = [lj]
fire.methods = [nve]

sim.operations.integrator = fire

In [4]:
sim.operations.writers.clear()

In [5]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)

logger = hoomd.logging.Logger(categories=["scalar"])

logger.add(thermodynamic_properties, ["potential_energy"])

writer = hoomd.write.GSD(hoomd.trigger.Periodic(40), filename='test_mimse.gsd', mode="wb", log=logger)

sim.operations.writers.append(writer)

table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=40),
                          logger=logger)

# sim.operations.writers.append(table)

In [6]:
sim.run(8000)

In [10]:
mimse_force._state.box.is2D

True

In [7]:
fire.forces.append(mimse_force)

In [8]:
for i in range(200):
    fire.reset()
    while not fire.converged:
        sim.run(40)
    print(lj.energy, mimse_force.energy)
    snap = sim.state.get_snapshot()
    mimse_force.add_bias(20.0, 1.0, snap.particles.position, np.zeros_like(snap.particles.position))

-793.8314932505945 0.0
-793.5345073836108 0.0010862838468739255
-793.658130597029 0.001439170046424465
-793.370436741933 0.00031033833238325315
-793.0771260974104 0.0033191512032004766
-794.6304847448923 0.0
-794.9393395626322 0.0014832819267240893
-794.4504397397271 0.0012442664542078319
-794.2792510408846 0.0038594020876558943
-794.0879280341812 0.005879252345692976
-794.3313960233116 0.003676066120092969
-794.4945579857598 0.007133507289154295
-794.118942357639 0.006956981098440451
-794.1347506192606 0.004765933030483893
-794.4639773144838 0.0017063961593777321
-794.1530103188328 0.002130654861858261
-794.1436031853195 0.00688354159883886
-793.9730558329696 0.00862428712544272
-794.0031127408979 0.006624962009524893
-794.4161891406475 0.0006229692832704359
-794.5965215581381 0.002872039663373016
-796.1780543887876 0.0
-799.3869314116848 0.0
-798.7845495373757 0.0017086431854096418
-798.9540333894072 0.0033360755313083246
-803.8688282408194 0.0
-803.7586613771664 3.9796045996354223e-

In [9]:
mimse_force.nn

[11, 12, 13]

In [17]:
lj.energy

-793.7219270695075

In [15]:
len(mimse_force.nn)

50

In [10]:
sim.run(0)

In [54]:
x = hoomd._hoomd.make_scalar3(1,10,0)
im = hoomd._hoomd.make_int3(0,0,1)
flags = hoomd._hoomd.make_char3(0,0,0)
y = sim.state.box._cpp_obj.wrap(x, im, flags)